# DMW_A7 : One Class Slab SVM

# Group Members:
  
  Milan Ashvinbhai Bhuva - IIT2018176 
  
  Manav Kamlesh Agrawal - IIT2018178 
  
  Mohammed Aadil - IIT2018179 
  
  Ankit Kumar Rauniyar - IIT2018202

# Importing libraries

In [1]:
"""Importing librarires"""
import pandas as pd
import numpy as np
import scipy as spy
import sklearn as skl
import seaborn as sns
import matplotlib.pyplot as plt
import math,random
import random as rnd

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import chi2_kernel as chi2
from cvxopt import solvers, lapack, matrix, spmatrix

from scipy.io import loadmat
from oc_slabSVM import OCC

# Data Visualisation

In [2]:
"""Importing dataset"""
dataset = pd.read_csv("letter-recognition.csv")
dataset2 = dataset.sort_values('lettr')
dataset2.head(15)

,lettr,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
19999,A,4,9,6,6,2,9,5,3,1,8,1,8,2,7,2,8
1027,A,3,6,5,4,1,7,5,3,1,6,1,8,2,7,2,7
11108,A,5,9,6,7,7,8,8,8,4,6,6,8,3,8,8,4
1030,A,4,10,7,7,5,7,5,2,3,5,2,6,3,7,4,4
19420,A,4,9,7,7,5,8,5,2,4,6,1,5,3,5,4,5
7566,A,4,9,7,6,3,12,2,4,3,11,1,9,3,7,3,9
7582,A,4,5,6,7,2,7,4,3,1,7,1,8,3,7,2,8
11094,A,3,2,5,3,2,6,2,2,2,5,2,8,2,6,2,6
1041,A,2,7,4,5,2,12,2,4,3,11,2,9,2,6,3,9
1042,A,3,4,6,6,2,9,3,3,3,8,2,9,3,6,3,9


# Data Pre processing

In [3]:
X = scale(np.array(dataset.iloc[:, 1:]))
y = np.array(dataset.iloc[:, 0])

print("X shape =", X.shape)
print("y shape =", y.shape)

y = np.asarray([ 1 if i == 'A' else -1 for i in y])

X shape = (20000, 16)
y shape = (20000,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 101)

print("X_train shape =", X_train.shape,"\nX_test shape  =", X_test.shape)
print("y_train shape =", y_train.shape,"\ny_test shape  =", y_test.shape)

X_train shape = (16000, 16) 
X_test shape  = (4000, 16)
y_train shape = (16000,) 
y_test shape  = (4000,)


In [5]:
'''linear kernel'''
clf = OCC.slabFitFunction(X_train,y_train,'linear')
y_pred_linear = OCC.slabPredictFunction(clf,X_test)
accuracy_linear = OCC.getAcc(y_pred_linear,y_test)
print('The linear kernel provides us with an accuracy of : ',accuracy_linear,'%')

'''RBF kernel'''
clf = OCC.slabFitFunction(X_train,y_train,'rbf')
y_pred_rbf = OCC.slabPredictFunction(clf,X_test)
accuracy_rbf = OCC.getAcc(y_pred_rbf,y_test)
print('The rbf kernel provides us with an accuracy of : ',accuracy_rbf,'%')

'''chi2 kernel'''
X_train2 = np.abs(np.asarray(X_train,dtype='float64'))
X_test2 = np.abs(np.asarray(X_test,dtype='float64'))
clf = OCC.slabFitFunction(X_train2,y_train,chi2)
y_pred_chi2 = OCC.slabPredictFunction(clf,X_test2)
accuracy_chi2 = OCC.getAcc(y_pred_chi2,y_test)
print('The chi^2 kernel provides us with an accuracy of : ',accuracy_chi2,'%')

'''Hellinger kernel'''
clf = OCC.slabFitFunction(X_train,y_train,'hellinger')
y_pred_hellinger = OCC.slabPredictFunction(clf,X_test)
accuracy_hellinger = OCC.getAcc(y_pred_hellinger,y_test)
print('The hellinger kernel provides us with an accuracy of : ',accuracy_hellinger,'%')

The linear kernel provides us with an accuracy of :  59.94999999999999 %
The rbf kernel provides us with an accuracy of :  95.575 %
The chi^2 kernel provides us with an accuracy of :  82.75 %
The hellinger kernel provides us with an accuracy of :  62.625 %


In [6]:
"""Accuracy array"""
acc_array = []
acc_array.append(str(1-float(accuracy_linear/100))[:4])
acc_array.append(str(1-float(accuracy_rbf/100))[:4])
acc_array.append('NaN')
acc_array.append(str(1-float(accuracy_chi2/100))[:4])
acc_array.append(str(1-float(accuracy_hellinger/100))[:4])


table = pd.DataFrame(columns=['Kernel', 'KPCA', 'SVDD', 'OCSVM', 'OCSSVM', 'KMS_SVM'])
table['Kernel'] = ['Linear', 'RBF', 'Intersection', 'Hellinger', 'χ2']
table['KPCA'] = ['0.02', '0.05', '0.18', '0.01', '0.18']
table['SVDD'] = ['0.09', '0.07', '0.01', '0.02', '0.00']
table['OCSVM'] = ['0.01', '0.14', '0.04', '0.02', '0.02']
table['OCSSVM'] = acc_array
table['KMS_SVM'] = ['0.04', '0.04', '0.22', '0.10', '0.17']
table

,Kernel,KPCA,SVDD,OCSVM,OCSSVM,KMS_SVM
0,Linear,0.02,0.09,0.01,0.40,0.04
1,RBF,0.05,0.07,0.14,0.04,0.04
2,Intersection,0.18,0.01,0.04,NaN,0.22
3,Hellinger,0.01,0.02,0.02,0.17,0.10
4,χ2,0.18,0.00,0.02,0.37,0.17
